In [ ]:
%pip install pandas
%pip install openpyxl
%pip install nltk


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

data = pd.read_excel('hasil/hasil_preposesing.xlsx')

In [ ]:
import csv

def read_lexicon(file_path):
    with open(file_path, 'r') as csvfile:
        lexicon = {row[0]: int(row[1]) for row in csv.reader(csvfile)}
    return lexicon

def sentiment_analysis_lexicon_indonesia(text, lexicon_positive, lexicon_negative):
    score = sum(lexicon_positive.get(word, 0) + lexicon_negative.get(word, 0) for word in text)
    if pd.isna(score):
        score = 0  # Set score to 0 if it's NaN
    unclassified_words = [word for word in text if word not in lexicon_positive and word not in lexicon_negative]
    return pd.Series([score, unclassified_words], index=['polarity_text', 'unclassified_words'])

lexicon_positive = read_lexicon('dataset/lexicon_positive.csv')
lexicon_negative = read_lexicon('dataset/lexicon_negative.csv')

def tokenize_and_analyze_sentiment(text, lexicon_positive, lexicon_negative):
    tokenized_text = word_tokenize(text)
    return sentiment_analysis_lexicon_indonesia(tokenized_text, lexicon_positive, lexicon_negative)


data[['polarity_text', 'unclassified_words']] = data['text_join'].fillna('').apply(lambda x: tokenize_and_analyze_sentiment(x, lexicon_positive, lexicon_negative))
data

In [ ]:
unclassified_words = set()

for value in data['unclassified_words']:
    if isinstance(value, list) and value:
        unclassified_words.update(value)

filtered_df = pd.DataFrame(list(unclassified_words), columns=['unclassified_words'])

#simpan data emoticon yang tidak bisa klasifikasi
filtered_df.to_excel('gagal_proses/unclassified_words.xlsx', index=False)

#hapus kolom unprocessable_emoticons tidak butuh
data.drop('unclassified_words', axis=1, inplace=True)

In [ ]:
data.to_excel('hasil/hasil_preposesing.xlsx', index=False)